In [1]:
import os
from sklearn import preprocessing
from collections import deque
import random
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM,CuDNNLSTM,BatchNormalization
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint

SEQ_LEN=60
FUTURE_PERIOD_PREDICT=3
RATIO_TO_PREDICT="LTC-USD"
EPOCHS=10
BATCH_SIZE=64
NAME=f"{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"
def classify(current,future):
    if float(future)>float(current):
        return 1
    else:
        return 0

    
def preprocess(df):
    df.drop('future',1)
    
    for col in df.columns:
        if col!="target":
            df[col]=df[col].pct_change()
            df.dropna(inplace=True)
            df[col]=preprocessing.scale(df[col].values)
    df.dropna(inplace=True)
    
    sequential_data=[]
    prev_days=deque(maxlen=SEQ_LEN)
    
    for i in df.values:
        prev_days.append([n for n in i[:-1]])
        if len(prev_days)==SEQ_LEN:
            sequential_data.append([np.array(prev_days),i[-1]])
    random.shuffle(sequential_data)
    
    buys=[]
    sells=[]
    
    for seq,target in sequential_data:
        if target==0:
            sells.append([seq,target])
        elif target==1:
            buys.append([seq,target])
    random.shuffle(buys)
    random.shuffle(sells)
    
    lower=min(len(buys),len(sells))
    
    buys=buys[:lower]
    sells=sells[:lower]
    
    sequential_data=buys+sells
    random.shuffle(sequential_data)
    
    X=[]
    y=[]
    
    for seq,target in sequential_data:
        X.append(seq)
        y.append(target)
    return np.array(X),y

In [2]:
import pandas as pd
df=pd.read_csv("LTC-USD.csv",names=["time","low","high","open","close","volumn"])
print(df.head(4))

         time        low       high       open      close      volumn
0  1528968660  96.580002  96.589996  96.589996  96.580002    9.647200
1  1528968720  96.449997  96.669998  96.589996  96.660004  314.387024
2  1528968780  96.470001  96.570000  96.570000  96.570000   77.129799
3  1528968840  96.449997  96.570000  96.570000  96.500000    7.216067


In [3]:
main_df=pd.DataFrame()

ratios=["BTC-USD","LTC-USD","ETH-USD","BCH-USD"]
for ratio in ratios:
    dataset=f"{ratio}.csv"
    df=pd.read_csv(dataset,names=["time","low","high","open","close","volumn"])
    #print(df.head(3))
    df.rename(columns={"close":f"{ratio}_close","volumn":f"{ratio}_volumn"},inplace=True)
    df.set_index("time",inplace=True)
    df=df[[f"{ratio}_close",f"{ratio}_volumn"]]
    
    if len(main_df)==0:
        main_df=df
    else:
        main_df=main_df.join(df)
    print(main_df.head(4))
    

            BTC-USD_close  BTC-USD_volumn
time                                     
1528968660    6489.549805        0.587100
1528968720    6487.379883        7.706374
1528968780    6479.410156        3.088252
1528968840    6479.410156        1.404100
            BTC-USD_close  BTC-USD_volumn  LTC-USD_close  LTC-USD_volumn
time                                                                    
1528968660    6489.549805        0.587100      96.580002        9.647200
1528968720    6487.379883        7.706374      96.660004      314.387024
1528968780    6479.410156        3.088252      96.570000       77.129799
1528968840    6479.410156        1.404100      96.500000        7.216067
            BTC-USD_close  BTC-USD_volumn  LTC-USD_close  LTC-USD_volumn  \
time                                                                       
1528968660    6489.549805        0.587100      96.580002        9.647200   
1528968720    6487.379883        7.706374      96.660004      314.387024   
152896

In [4]:
main_df['future']=main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT)
print(main_df[[f"{RATIO_TO_PREDICT}_close","future"]].head(3))

            LTC-USD_close     future
time                                
1528968660      96.580002  96.500000
1528968720      96.660004  96.389999
1528968780      96.570000  96.519997


In [5]:
main_df['target']=list(map(classify,main_df[f"{RATIO_TO_PREDICT}_close"],main_df["future"]))
print(main_df[[f"{RATIO_TO_PREDICT}_close","future","target"]].head(3))

            LTC-USD_close     future  target
time                                        
1528968660      96.580002  96.500000       0
1528968720      96.660004  96.389999       0
1528968780      96.570000  96.519997       0


In [6]:
times=sorted(main_df.index.values)
last=times[-int(0.05*len(times))]

validation_main_df=main_df[(main_df.index>=last)]
main_df=main_df[(main_df.index<last)]

train_X,train_y=preprocess(main_df)
validation_X,validation_y=preprocess(validation_main_df)



In [8]:
model=Sequential()
model.add(CuDNNLSTM(128,input_shape=(train_X.shape[1:]),return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(CuDNNLSTM(128,input_shape=(train_X.shape[1:]),return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(CuDNNLSTM(128,input_shape=(train_X.shape[1:]),return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32,activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(2,activation="softmax"))

opt=tf.keras.optimizers.Adam(lr=0.001,decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

tensorboard=TensorBoard(log_dir=f'logs/{NAME}')

filepath='RNN_FINAL-{epoch:02d}-{val_acc:.3f}'
checkpoint=ModelCheckpoint("models/{}.model".format(filepath,monitor="val_acc",verbose=1,save_best_only=True,mode='max'))

history=model.fit(
    train_X,train_y,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(validation_X,validation_y)
    
)
#callbacks=[tensorboard,checkpoint]

Train on 69186 samples, validate on 3056 samples
Epoch 1/10


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: Incompatible shapes: [64,60] vs. [64]
	 [[{{node metrics_2/acc/Equal}}]]
	 [[loss_1/mul/_521]]
  (1) Invalid argument: Incompatible shapes: [64,60] vs. [64]
	 [[{{node metrics_2/acc/Equal}}]]
0 successful operations.
0 derived errors ignored.